In [1]:
from typing import TypedDict, Annotated, Optional
from langgraph.graph import add_messages, StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4
import json

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [2]:
search_tool = TavilySearch(max_results=4)

tools = [search_tool]

memory = MemorySaver()

In [3]:
llm_with_tools = model.bind_tools(tools=tools)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

class State(TypedDict):
    messages: Annotated[list, add_messages]

async def model(state: State):
    result = await llm_with_tools.ainvoke(state["messages"])
    return {
        "messages": [result], 
    }

async def tools_router(state: State):
    last_message = state["messages"][-1]

    if(hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0):
        return "tool_node"
    else: 
        return END
    
# Can us ToolNode function instead
async def tool_node(state):
    """Custom tool node that handles tool calls from the LLM."""
    # Get the tool calls from the last message
    tool_calls = state["messages"][-1].tool_calls
    
    # Initialize list to store tool messages
    tool_messages = []
    
    # Process each tool call
    for tool_call in tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_id = tool_call["id"]
        
        # Handle the search tool
        if tool_name == "tavily_search_results_json":
            # Execute the search tool with the provided arguments
            search_results = await search_tool.ainvoke(tool_args)
            
            # Create a ToolMessage for this result
            tool_message = ToolMessage(
                content=str(search_results),
                tool_call_id=tool_id,
                name=tool_name
            )
            
            tool_messages.append(tool_message)
    
    # Add the tool messages to the state
    return {"messages": tool_messages}

graph_builder = StateGraph(State)

graph_builder.add_node("model", model)
graph_builder.add_node("tool_node", tool_node)
graph_builder.set_entry_point("model")

graph_builder.add_conditional_edges("model", tools_router)
graph_builder.add_edge("tool_node", "model")

graph = graph_builder.compile(checkpointer=memory)

In [7]:
config = {
    "configurable": {
        "thread_id": 5
    }
}

response = await graph.ainvoke({
    "messages": [HumanMessage(content="What is my name?")], 
}, config=config)

response

{'messages': [HumanMessage(content='How are you doing?', additional_kwargs={}, response_metadata={}, id='cfc60098-453b-418e-90f2-9be5d35681a4'),
  AIMessage(content='I am doing well, thank you for asking! I am ready to assist you with any questions or tasks you may have.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--affe2fa6-5eec-4cab-9f41-0949e30a243e-0', usage_metadata={'input_tokens': 1542, 'output_tokens': 25, 'total_tokens': 1567, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='My name is Zak', additional_kwargs={}, response_metadata={}, id='d7120e56-90c0-4dbe-be0b-76cd27ef0844'),
  AIMessage(content="It's nice to meet you, Zak! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings':

In [13]:
config = {
    "configurable": {
        "thread_id": 7
    }
}

# Use async for to iterate over the async generator
async for event in graph.astream_events({
    "messages": [HumanMessage(content="Give me 100 word essay on climate change")]
}, config=config, version="v2"):
    if event["event"] == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

[{'type': 'text', 'text': "Climate change, a pressing global crisis, refers to long-term shifts in temperatures and weather patterns. Primarily driven by human activities, especially the burning of fossil fuels, it leads to an increase in greenhouse gases trapping heat in the Earth's atmosphere.", 'extras': {'signature': 'CikB0e2Kby4ubJj39JHFL1AqcS7L/nZMVBZllulk4HbX/l0kG2fN7HVvkgpYAdHtim8VeaxluxV3SdIRiJCC5dY1YR947m/gEgedBg9sT/b9WcjnBjbZKg0Axj3SlIZWiwJpQ8616A01uxeo99h0fOad4IOJRtwdbmkopPdlhOh48z8QjwqhAQHR7YpvFvSlRfQ9Eib/QhWalGh9y4Mf3AsuC/I9E60P9xK3bvbczfaBmINwoov30nYZ5CppWqPBTpweF5vKUy1PN3rt+WQbZutq88ASLaYc3o2N65ix0P7VJKeS4hNm/cTmjZs21oYJb0WOcYzpPMaUc7CfZzVyfjE8UNCGhES170cUE3Bwv1FV0ZRVF2GTUkJMntCZ45wk/PD9Tf7WwSux'}}] The consequences are far-reaching: rising sea levels, extreme weather events, disruptions to ecosystems, and threats to food security. Addressing this challenge requires urgent collective action, including transitioning to renewable energy sources, promoting sustainable prac